$Notes$
1. tensor jup: base [0] = (1,0) and site count start from 0.
2. tensor jup correct base: [0] = (0,1) and site count start from 1

$Functions$

1. quvec(state) - takes a state and represent it as Z base vector
2. ket(vec) - takes a vector and represent it as a Ket
3. operate(operators, psi) - operate a list of operators on a state
4. n_d_gates(operators) - tensor product a list of operators by theis order the list
5. matrix_gate(gates,quvec) - gates is a list of gates and theis indices, i.e [X,0,Y,2], compose a matrix in the size of the state with the given gates
5. general_operate(gates,qvec) - gates is a list of gates and theis indices, i.e [X,0,Y,2] will apply X on the 0th qubit and Y on the 2nd of the state
    
6. norm(state) - normalize a state
7. NOT IN CODE. measure_2d(state) - recievve a 1d state and collapses it randomally to an eigenstate of Z
8. measure_probability(state) - recieve super position of Z base and outputs only one eigenstate 
    randomlly
    chosen with probability bias
9. tensor_exponent(tensor,power) - gets a tensor and returns the tensor product done with it self for "power" 
    times
10. NOT IN CODE. index vector(spin,location,dimension) - return the superposition of states in Z base with the given    
    dimension that has the given spin in location
    
11. NOT IN CODE. collapse(spin,location,state) - collapse state to only the eigenstates with the pointed spin and location
12. NOT IN CODE bias_coin(A , B , p) - choose randomly between A and B with a bias p. as p grows, chance for A grows
13. observation_value(operator, state) - operator observation value
14. projector(operator, eigenvalue) - returns the + or - projector of clifford gate
15. Measure_2d(operator, state) - performs the clifford gate measurement on a state, return collapsed state and
    proobability for +1 eigenvalue
16. Measure(operator, state, site) - for any dimension, operator measurement on specific gate
17. NOT IN CODE. measure_simultaneously_seperate(measureables, state) - gets list of operators and sites i.e [X,1,Y,2] and measure the outcome,  of a state by taking expeectation value for each site seperately.
18. measure_simultaneously2(measureables, state) - taknig the expectation value for the whole operator and measures it.
19. Measure_Z_for_entire_state(state, probability) - series of Z measurements for each site at a time, probability for measuring a site is given.
20. Measure_XX_for_entire_state(state, probability) - like for Z but measure XX for each neighboring sites.
21. sequence(state, probability, runs) - apply measure Z and XX for entire state measurements, repeats the process as given in runs.

$To Do's$
 
- run codes for more time, store the data for each run in another file.
- smooth plots - more runs and trajectories, as higher systems I can have
- plot expectation value on Z for each site for a single trajectory for each time.

$Questions$


In [ ]:
#7
def measure_2d(state):
    N_state = norm(state) #normalize the given state
    P_0 = np.abs(N_state[0])
    r = random.random()
    if r < (P_0)**2:
        N_state[1] = 0
    else:
        N_state[0] = 0
    return norm(N_state)


In [ ]:
# 12
def bias_coin(A , B , p):
    r = random.random()  # Generates a random float between 0 and 1
    if r < p:
        return A
    else:
        return B


In [ ]:
#19
def measure_simultaneously_seperate(measureables, state): #measureables are operators and sites. i.e [X,1,Y,3]
    state = norm(state)
    gates = [] # 2d clifford gates
    sites = [] # sites list
    operators = [] # gates working on each site
    dimension  = int(np.log2(len(state)))
    i = 0
    while i < len(measureables): 
        gates.append(measureables[i])
        site = measureables[i+1]
        sites.append(site)

        O_l = n_d_gates([I]*(site - 1) + [measureables[i]] + [I]*(dimension-site))
        operators.append(O_l)
        i += 2

    probabilities = [(observation_value(i,state) + 1)/2 for i in operators]
    eigenvalues = [random.choices([1,-1], cum_weights = [p,1], k = 1) for p in probabilities]
    # sub_projectors = [(I + eigenvalues*i)/2 for i in gates]
    sub_projectors = sum([[(I + eigenvalues[i]*gates[i])/2, sites[i]] for i in range(len(gates))], [])
    Projected = norm(general_operate(sub_projectors, state))
    # return sub_projectors, eigenvalues, probabilities, gates, sites
    return Projected,  sub_projectors, eigenvalues, probabilities, gates, sites
    



In [ ]:

def index_vector(spin,location,dimension): 
    plus = [1,1]
    index_psi0 = np.kron(tensor_exponent(plus,location), quvec([spin]))
    index_psi = np.kron(index_psi0, tensor_exponent(plus,dimension-1-location))
    return index_psi

def collapse(spin,location,state): #spin is a number (0 or 1), location is the wanted place to measure spin from 0 to n-1, state in vector form
    dimension = np.log2(len(state))
    index_psi = index_vector(spin,location,dimension)
    probability_vec = index_psi * norm(state)
    probability = np.matmul(np.conj(probability_vec), probability_vec)
    collapsed = norm(measure_probability(probability_vec))
    return probability, norm(probability_vec), collapsed
